In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/english-abusive-MuRIL")
model = AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/english-abusive-MuRIL")

In [2]:
save_directory = "./local_hate_speech_model"
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")

Model and tokenizer saved to ./local_hate_speech_model


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
local_directory = "./local_hate_speech_model"

tokenizer = AutoTokenizer.from_pretrained(local_directory)
model = AutoModelForSequenceClassification.from_pretrained(local_directory)

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [4]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

file_path = "labeled_data.csv" 
df = pd.read_csv(file_path)


In [5]:
df['label'] = df['class'].apply(lambda x: 1 if x in [0, 1] else 0)
df = df[['tweet', 'label']].rename(columns={'tweet': 'conversation'})

_, val_texts, _, val_labels = train_test_split(
    df['conversation'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)


tokenizer = AutoTokenizer.from_pretrained("./local_hate_speech_model")
model = AutoModelForSequenceClassification.from_pretrained("./local_hate_speech_model")

In [6]:

val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=64, return_tensors="pt")

val_inputs = val_encodings["input_ids"]
val_masks = val_encodings["attention_mask"]
val_labels = torch.tensor(val_labels)

batch_size = 8
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

device = torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(197285, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [7]:
# Evaluation (no training)
model.eval()
total_correct = 0
total_samples = 0
with torch.no_grad():
    for batch in val_loader:
        batch_inputs, batch_masks, batch_labels = [b.to(device) for b in batch]
        outputs = model(input_ids=batch_inputs, attention_mask=batch_masks)
        predictions = torch.argmax(outputs.logits, dim=1)
        total_correct += (predictions == batch_labels).sum().item()
        total_samples += batch_labels.size(0)

accuracy = total_correct / total_samples
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.9671


In [8]:
def predict(text):
    model.eval()
    encoding = tokenizer(text, truncation=True, padding=True, max_length=64, return_tensors="pt")
    input_ids, attention_mask = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    
    return "Offensive" if prediction == 1 else "Not Offensive"

In [9]:
print(predict("You are an idiot"))
print(predict("You are an bitch"))
print(predict("Hope you have a great day!"))
print(predict("You are a good guy"))
print(predict("You are"))
print(predict("You are a "))
print(predict("You are a poor guy"))
print(predict("You are ugly"))
print(predict("You are preety"))
print(predict("Fuck you"))
print(predict("Suck it"))

Offensive
Offensive
Not Offensive
Not Offensive
Not Offensive
Not Offensive
Not Offensive
Offensive
Not Offensive
Offensive
Not Offensive
Offensive
